# --------------------------------------------------------

## Step 1) Import Libraries

In [2]:
#Libraries
import os
import shutil
import pandas as pd
import kagglehub


c:\Program Files\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# --------------------------------------------------------

## Step 2) Download Dataset

the following script downloads and moves the dataset to root/Data/Raw

set to `False` if you already have the dataset

The incoming data for sales only holds sales for jan - feb, the sales forecasting models resolution will be on a weekly or daily scale

In [3]:
RUN_DOWNLOAD = False

In [4]:
if RUN_DOWNLOAD:
    # Download the dataset
    download_path = kagglehub.dataset_download("bhanupratapbiswas/inventory-analysis-case-study")

    # Define the target directory
    target_dir = os.path.abspath("../../Data/Raw")

    # Make sure the target directory exists
    os.makedirs(target_dir, exist_ok=True)

    # Move all files (not folders) from download_path to target_dir
    for filename in os.listdir(download_path):
        src = os.path.join(download_path, filename)
        dst = os.path.join(target_dir, filename)

        if os.path.isfile(src):
            shutil.move(src, dst)

    print("Files moved to:", target_dir)
else:
    print("Download block skipped.")

Download block skipped.


# --------------------------------------------------------

## Step 3) Create Clean CSV files

Start pulling data from dataset to produce clean csv files ready for the inventory ledger

In [5]:
# Set file paths
raw_path = '../../Data/Raw/'
processed_path = '../../Data/Processed/'

# Create processed directory if it doesn't exist
os.makedirs(processed_path, exist_ok=True)

In [6]:
# Function to split composite InventoryId
def split_inventory_id(df):
    split_cols = df['InventoryId'].str.split('_', expand=True)
    df['Store'] = split_cols[0]
    df['Location'] = split_cols[1]
    df['ItemId'] = split_cols[2]
    df.drop(columns=['InventoryId'], inplace=True)
    return df

In [22]:
# Create Sales.csv (Store, ItemId, SalesQuantity, SalesDate)
# Load the Sales.csv file
sales_file = os.path.join(raw_path, 'SalesFINAL12312016.csv')
sales_df = pd.read_csv(sales_file)

# Clean Sales.csv
# 1. Keep only relevant columns (no need for Location, it's handled in Stores.csv)
sales_cleaned = sales_df[[
    'InventoryId',
    'Store',
    'SalesQuantity',
    'SalesDate'
]]

# 2. Strip whitespace and filter for stores "1" and "2"
sales_cleaned['Store'] = sales_cleaned['Store'].astype(str).str.strip()
sales_cleaned = sales_cleaned[sales_cleaned['Store'].isin(['1', '2'])]

# Apply split function to separate the composite InventoryId
sales_cleaned = split_inventory_id(sales_cleaned)

# Drop the Location column after splitting the InventoryId
sales_cleaned.drop(columns=['Location'], inplace=True)

# Rename 'Store' column to 'StoreID'
sales_cleaned.rename(columns={'Store': 'StoreID'}, inplace=True)

# Save the cleaned Sales.csv
sales_cleaned.to_csv(os.path.join(processed_path, 'Sales.csv'), index=False)

sales_cleaned.head()


C:\Users\edcul\AppData\Local\Temp\ipykernel_17924\3327469519.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales_cleaned['Store'] = sales_cleaned['Store'].astype(str).str.strip()


,StoreID,SalesQuantity,SalesDate,ItemId
0,1,1,1/1/2016,1004
1,1,2,1/2/2016,1004
2,1,1,1/3/2016,1004
3,1,1,1/8/2016,1004
4,1,2,1/9/2016,1005


In [21]:
# Create Purchases.csv (Store, PODate, ReceivingDate, Quantity)
# Load the Purchases.csv file
purchases_file = os.path.join(raw_path, 'PurchasesFINAL12312016.csv')
purchases_df = pd.read_csv(purchases_file)

# Clean Purchases.csv
# 1. Keep only relevant columns
purchases_cleaned = purchases_df[[
    'InventoryId',
    'Store',
    'PODate',
    'ReceivingDate',
    'Quantity'
]]

# 2. Strip whitespace and filter for stores "1" and "2"
purchases_cleaned['Store'] = purchases_cleaned['Store'].astype(str).str.strip()
purchases_cleaned = purchases_cleaned[purchases_cleaned['Store'].isin(['1', '2'])]

# Apply split function
purchases_cleaned = split_inventory_id(purchases_cleaned)

# Drop the Location column after splitting the InventoryId
purchases_cleaned.drop(columns=['Location'], inplace=True)

# Rename 'Store' column to 'StoreID'
purchases_cleaned.rename(columns={'Store': 'StoreID'}, inplace=True)

# Save the cleaned Purchases.csv
purchases_cleaned.to_csv(os.path.join(processed_path, 'Purchases.csv'), index=False)

purchases_cleaned.head()


C:\Users\edcul\AppData\Local\Temp\ipykernel_17924\2386600434.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  purchases_cleaned['Store'] = purchases_cleaned['Store'].astype(str).str.strip()


,StoreID,PODate,ReceivingDate,Quantity,ItemId
3,1,2015-12-22,2016-01-01,6,5255
6,1,2015-12-20,2016-01-01,12,8358
9,1,2015-12-20,2016-01-01,23,4233
10,1,2015-12-20,2016-01-01,6,3830
40,1,2015-12-20,2016-01-01,11,4670


In [23]:
# Create OpeningStock.csv (Store, Location, ItemId, onHand, startDate)
# Load the beginning inventory file
OpeningStock_file = os.path.join(raw_path, 'BegInvFINAL12312016.csv')
OpeningStock_df = pd.read_csv(beg_inv_file)

# 🧹 Clean BegInvFINAL12312016.csv
# 1. Keep only relevant columns
OpeningStock_Cleaned = OpeningStock_df[[
    'InventoryId',
    'Store',
    'onHand',
    'startDate'
]]

# 2. Strip whitespace from Store and keep only store "1" and "2"
OpeningStock_Cleaned['Store'] = OpeningStock_Cleaned['Store'].astype(str).str.strip()
OpeningStock_Cleaned = OpeningStock_Cleaned[OpeningStock_Cleaned['Store'].isin(['1', '2'])]

# 3. Replace the startDate with 2015-12-31
OpeningStock_Cleaned['startDate'] = '2015-12-31'

# Apply split function
OpeningStock_Cleaned = split_inventory_id(OpeningStock_Cleaned)

# Drop the Location column after splitting the InventoryId
OpeningStock_Cleaned.drop(columns=['Location'], inplace=True)

# Rename 'Store' column to 'StoreID'
OpeningStock_Cleaned.rename(columns={'Store': 'StoreID'}, inplace=True)

# Save the cleaned beginning inventory
OpeningStock_Cleaned.to_csv(os.path.join(processed_path, 'OpeningStock.csv'), index=False)

OpeningStock_Cleaned.head()


C:\Users\edcul\AppData\Local\Temp\ipykernel_17924\2026519689.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  OpeningStock_Cleaned['Store'] = OpeningStock_Cleaned['Store'].astype(str).str.strip()


,StoreID,onHand,startDate,ItemId
0,1,8,2015-12-31,58
1,1,7,2015-12-31,60
2,1,6,2015-12-31,62
3,1,3,2015-12-31,63
4,1,6,2015-12-31,72


In [25]:
# Create Inventory.csv (ItemID, Description)
# Pull ItemID and Description from original CSVs
sales_info = sales_df[['Brand', 'Description']].rename(columns={'Brand': 'ItemID'})
purchases_info = purchases_df[['Brand', 'Description']].rename(columns={'Brand': 'ItemID'})
opening_info = OpeningStock_df[['Brand', 'Description']].rename(columns={'Brand': 'ItemID'})

# Combine and deduplicate
inventory_df = pd.concat([sales_info, purchases_info, opening_info])
inventory_df.drop_duplicates(subset=['ItemID', 'Description'], keep='first', inplace=True)

# Save Inventory Master file
inventory_df.to_csv(os.path.join(processed_path, 'Inventory.csv'), index=False)

inventory_df.head()


,ItemID,Description
0,1004,Jim Beam w/2 Rocks Glasses
4,1005,Maker's Mark Combo Pack
8,10058,F Coppola Dmd Ivry Cab Svgn
17,1006,Jim Beam Candy Cane 4/50mLs
18,10062,Terroirs du Rhone


In [ ]:
# Extract InventoryID from a few datasets to make sure all inventory items are read

beg_inv_file = os.path.join(raw_path, 'BegInvFINAL12312016.csv')
tbl1df = pd.read_csv(beg_inv_file)

purchases_file = os.path.join(raw_path, 'PurchasesFINAL12312016.csv')
tbl2df = pd.read_csv(purchases_file)

tbl1 = tbl1df['InventoryId'].dropna().unique()
tbl2 = tbl2df['InventoryId'].dropna().unique()

# Combine and deduplicate InventoryIDs
ALLtbls = pd.Series(list(set(tbl1).union(set(tbl2))))

# Filter only those entries that can be cleanly split into 3 parts
ALLtbls_valid = ALLtbls[ALLtbls.str.count('_') == 2]

# Split InventoryID into StoreID, Location, and ItemID
stores_split = ALLtbls_valid.str.split('_', expand=True)
stores_split.columns = ['StoreID', 'Location', 'ItemID']

# Keep only stores 1 and 2
stores_split = stores_split[stores_split['StoreID'].isin(['1', '2'])]

# Create Stores DataFrame and drop duplicates
stores_df = stores_split[['StoreID', 'Location']].drop_duplicates()

# Save to CSV
stores_df.to_csv(os.path.join(processed_path, 'Stores.csv'), index=False)

stores_df.head()


,StoreID,Location
5,1,HARDERSFIELD
305,2,ASHBORNE
